### Парсинг файлов после MAST 

In [1]:
from bs4 import BeautifulSoup

In [3]:
with open("mast_interpro_1_cl.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

In [4]:
motifs = soup.motifs.findAll('motif')
length = [motif['length'] for motif in motifs]
length

['35', '31', '29']

На 24 месте в первом мотиве не должен стоять Cys. Проверим это и запишем проверенные сиквенсы в отдельные файл.

In [5]:
sequences = soup.sequences.findAll('sequence')

In [6]:
import re
reg = re.compile('[^a-zA-Z ]')

In [ ]:
Cys_seq_names = set()
Cys_seq = set()
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True:
            if reg.sub('', seg.data.string)[int(seg.find(idx = '0')['pos']) - int(seg['start']) + 23] != 'C':
                Cys_seq_names.add(seq['name'])
                Cys_seq.add(seq)

In [ ]:
with open('Cysless_sequences_4.xml', 'w') as f:
    for i in Cys_seq:
        f.write(str(i) + '\n')

Отдельно запишем данные в формате fasta

In [ ]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [ ]:
new_file = []
for name in Cys_seq_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [ ]:
with open('Cysless_proteins_4.fasta', 'w') as f:
    f.write(''.join(new_file))import pandas as pd

### Парсинг результатов pfam

In [7]:
import pandas as pd

Переведем txt таблицу в удобный нам пандас.

In [127]:
columns = []
d = []
with open('pfam_1_cl.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

In [128]:
data = pd.DataFrame(d, columns=columns)
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,A0A126WYI0,50,143,42,146,PF13426.7,PAS_9,Domain,9,101,104,69.2,3e-19,1,CL0183
1,A0A126WYI0,282,385,280,386,PF13426.7,PAS_9,Domain,3,103,104,76.4,1.8e-21,1,CL0183
2,A0A126X452,22,127,21,128,PF13426.7,PAS_9,Domain,2,103,104,55.4,6.1e-15,1,CL0183
3,A0A126X452,244,341,241,345,PF13426.7,PAS_9,Domain,4,98,104,62.6,3.3e-17,1,CL0183
4,A0A126WXX1,41,131,34,134,PF13426.7,PAS_9,Domain,11,100,104,61.2,9.5e-17,1,CL0183


### Идентифекация LOV 

Отбор по отсутствию Cys уже прошел, поэтому необходимо отсеять по p-value. 

In [116]:
with open("Cysless_sequences_4.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

Отобрали dictionary нужных наименований со списком позиций трех мотивов (этих троек мб несколько в последовательности)

In [117]:
sequences = soup.sequences.find_all('sequence')
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if (bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True and bool(seg.find(idx = '2')) == True):
            hits = seg.find_all('hit')
            if (float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15 and float(hits[2]['pvalue']) <= 1e-15):
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos']), 
                                                         int(hits[2]['pos'])])

In [118]:
new_names = []
wrong = []
for k in seqs.keys():
    seq_data = data.loc[data['seq id'] == k] # data - таблица с результатами pfam, здесь отсев по id
    df = seq_data.loc[seq_data['clan'] == 'CL0183'] # здесь отсев по id клана (группы) домена PAS
    b = False
    for hit in seqs[k]:
        for i in range(df.shape[0]):
            if int(df.iloc[i, 3]) - hit[0] <= 2 and hit[2] + 28 - int(df.iloc[i, 4]) <= 10:
                b = True
    if not b:
        data.drop(data[data['seq id'] == k].index, inplace=True)
        wrong.append(k)
    else:
        new_names.append(k)   

In [119]:
with open('/home/dashik/projects/181015_conf_mipt_18/protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [120]:
new_file = []
for name in new_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [121]:
with open('final_proteins_4_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

Проверим на пересечения

In [122]:
with open('/home/dashik/projects/181015_conf_mipt_18/With_Cys/final_proteins_4.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [123]:
with open('final_proteins_4_cl.fasta', 'r') as f:
    fasta_cl = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta_cl.setdefault(name, []).append(line)

In [124]:
repeat = list(fasta.keys() & fasta_cl.keys())
len(repeat)

77

In [125]:
for r in repeat:
    fasta_cl.pop(r)

In [126]:
new_file = []
for name in fasta_cl.keys():
    new_file.append(name + '\n')
    for s in fasta_cl.get(name):
        new_file.append(s + '\n')
with open('final_proteins_4_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

Удалили повторы и объеденили финальные последоватеьнсти в оди файл, для построения дерева и выравнивания белковых последовательностей.

In [135]:
 # data - df pfam
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt
from tqdm import trange

In [134]:
def kingdom(s):
    handle = ExPASy.get_sprot_raw(s[0])
    try:
        record = SwissProt.read(handle)
        return record.organism_classification[0] + ' ' + record.organism_classification[1]
    except ValueError:
        return None

In [131]:
data['kingdom'] = None

In [132]:
data.shape

(4778, 16)

In [136]:
for i in trange(6110):
    data.iloc[i, 15] = kingdom(data.iloc[i,:])

  0%|          | 0/6110 [00:00<?, ?it/s]


HTTPError: HTTP Error 511: Network Authentication Required